In [1]:
%cd ..

/home/mori/work/adk-crash-course


In [2]:
from dotenv import load_dotenv
from google.adk.runners import Runner
from google.adk.sessions import DatabaseSessionService
from memory_agent.agent import memory_agent

In [3]:
load_dotenv()

True

In [4]:
# 記憶を永続化するのにSQLiteデータベースを使う
db_url = "sqlite:///./my_agent_data.db"
session_service = DatabaseSessionService(db_url=db_url)

In [5]:
# 初期状態を定義
initial_state = {
    "user_name": "もりさん",
    "reminders": [],
}

In [6]:
APP_NAME = "Memory Agent"
USER_ID = "mori"

In [7]:
existing_session = await session_service.list_sessions(
    app_name=APP_NAME,
    user_id=USER_ID,
)
print(existing_session)

if existing_session and len(existing_session.sessions) > 0:
    # 既存のセッションを続行
    SESSION_ID = existing_session.sessions[0].id
    print("Continuing existing session: {SESSION_ID}")
else:
    # 新しいセッションを開始
    # この時点でデータベースファイルが作られる
    new_session = await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        state=initial_state,
    )
    SESSION_ID = new_session.id
    print(f"Created new session: {SESSION_ID}")

sessions=[]
Created new session: c4e6d883-14d7-40ff-81e7-96539cf15ea5


In [8]:
new_session

Session(id='c4e6d883-14d7-40ff-81e7-96539cf15ea5', app_name='Memory Agent', user_id='mori', state={'user_name': 'もりさん', 'reminders': []}, events=[], last_update_time=1754613371.0)

In [9]:
# エージェントを実行するRunnerを作成
runner = Runner(
    agent=memory_agent,
    app_name=APP_NAME,
    session_service=session_service,
)
runner

In [27]:
from google.genai import types

query = "リマインダーはある？"
user_input = types.Content(role="user", parts=[types.Part(text=query)])

async for event in runner.run_async(
    user_id=USER_ID,
    session_id=SESSION_ID,
    new_message=user_input,
):
    print("*** is_final_response:", event.is_final_response())
    if event.content and event.content.parts:
        for part in event.content.parts:
            print(part)
    print("=" * 100)

*** is_final_response: False
video_metadata=None thought=None inline_data=None file_data=None thought_signature=b"\n\xd8\x01\x01T\xa8\\\xee\x8b\x1aT<\xe8\xc9\xa0\x9c\xe5a\xb4WI\xc1\xbe\xf6\xf1_\xd0*\xd7k7f\x99\x86\x81T#\x0cC\xd0\xab\xa7{\x04hX\xd4\x90 \xd6\x1f\x9dN i,\x81\xf2\x97U\xaee\xd3\xdc0\x1av\x9bV\x05S\xa7\xa0\xe3\xd2\x92-a\xe2\x15\xd6r\x1a*\x8f\xd8\x8c9\x15\xbd\xdeE\xe4\x17\xc5/$\x01=t>Ug\xce\xc8\x7f\x08\x19v\x1f%R\x07\xe7\xc5!\xf3\x17\xf22Z\xee\x03\x95\xad\x1b\xb6\x1a\xb1\x9f\xa0>\xee\xb4\xaa\xc7\x04\xbb?\xafH\x1a\xa8\xe73hu\x93\xdf\xcb/\x8ap\xb5\x88\xe0\xf3\xb8\xd3\xc4Z1E\x955\xcf\x96\x1a2x\xbea\xfe?Ih\xb6\x9ek\x9f\xf4\x9dU\xa7k\x8e\xc0\x9c\x1e_}X\xebP\x16[Y\xd3\x1b\xffqs\xe5`\xeey9\xecW\x8b\xf1'`\xf7\xbe\x8ayBn" code_execution_result=None executable_code=None function_call=FunctionCall(
  args={},
  id='adk-a404cfe0-76a1-40db-ae4f-383418df96d3',
  name='view_reminders'
) function_response=None text=None
*** is_final_response: False
video_metadata=None thought=None inline_dat

In [28]:
query = "牛乳を買うを追加して"
user_input = types.Content(role="user", parts=[types.Part(text=query)])

async for event in runner.run_async(
    user_id=USER_ID,
    session_id=SESSION_ID,
    new_message=user_input,
):
    print("*** is_final_response:", event.is_final_response())
    if event.content and event.content.parts:
        for part in event.content.parts:
            print(part)
    print("=" * 100)

*** is_final_response: False
video_metadata=None thought=None inline_data=None file_data=None thought_signature=b'\n\xa5\x01\x01T\xa8\\\xee\xa6;\x1cO\x04G\x1d\xd3\xd3\x07\xcd\xd7\x85#8\x91\xfa\x9f@8XR\xdb\xf6\xb0\x1f\xff\x1d\xcd\xf6\x19\x9d\xdb3OB\x15c\x16%\xf5\x83\xe0\x14MwGG\xea\xff\xbf\x90F<~\xda\x92\xf1\xa1\x88\xeb|^\xff\xff\x9fd\xe9\xe2\x80\xcf,a\x05\x9d\x8as_\xaa\xf1\xad\xd5n\xe9\x84\xa86\x8e\x87\xafGZ6FH!G\\\xaf\xbe\xefd\x8f\x1f\xf7.\x8f\x10\xea\x86\x1f\xb4\xe2=\\\xc5Jl\x9b\x14(i\xc5\xeaT8\xdd\xdf\xd1$`V\x16\xd0B(=\x11\xfd\x90\xa4p!\x00t\x04\x1f\x00"N\x95\xbae\xdbN+t;\x9a\xe0' code_execution_result=None executable_code=None function_call=FunctionCall(
  args={
    'reminder': '牛乳を買う'
  },
  id='adk-740d30b9-10d0-4d65-b9e0-6f11585eb39e',
  name='add_reminder'
) function_response=None text=None
*** is_final_response: False
video_metadata=None thought=None inline_data=None file_data=None thought_signature=None code_execution_result=None executable_code=None function_call=None funct

In [29]:
query = "ADKを使ってみるを追加して"
user_input = types.Content(role="user", parts=[types.Part(text=query)])

async for event in runner.run_async(
    user_id=USER_ID,
    session_id=SESSION_ID,
    new_message=user_input,
):
    print("*** is_final_response:", event.is_final_response())
    if event.content and event.content.parts:
        for part in event.content.parts:
            print(part)
    print("=" * 100)

*** is_final_response: False
video_metadata=None thought=None inline_data=None file_data=None thought_signature=b"\n\xa5\x01\x01T\xa8\\\xee\xe1\xe4\x1a\xae\xfc9Q\xe6\x82\xee\xe9,9='rzO\xdd\x01n!Vxet\x1cGO\t\xdd\x8dH\xad\xa8_U1\xf1\x06^\xd6/\xa7t[\xcd\x9f\x91 %\xd1\xd7H\xee\xe3E\x8f\x02\x15\x1b\x0fn \xc2\x15\xf96\xb7\xe5\xa8\xdc\x1c\x89\x84\xe3\xd219\x0b\x8e[\xc5\xd2\x9b\xeb\xc6\xd8 a\xa7\x93\xb2\xc9J\xb8l\xa2\x13m\xe8\x84:\x90\x0c\x8d4\xa9\xecH=YR\xf0\x0eP\x9e\x0e\xcdJ\x83\xd2\x85\xa4\x94\xf7\xb7G~\xb77\xb98\x80\xd5\x85\xf6\r\xbe\xa5M\xac\x99-\x81G\x11\xbfu\xe8)\xb6\xde3\xf5-/\r\x14\xee" code_execution_result=None executable_code=None function_call=FunctionCall(
  args={
    'reminder': 'ADKを使ってみる'
  },
  id='adk-b639e683-e4b4-4c0a-bada-ac1b05b729a2',
  name='add_reminder'
) function_response=None text=None
*** is_final_response: False
video_metadata=None thought=None inline_data=None file_data=None thought_signature=None code_execution_result=None executable_code=None function_call=No

In [30]:
query = "牛乳を買うは削除して"
user_input = types.Content(role="user", parts=[types.Part(text=query)])

async for event in runner.run_async(
    user_id=USER_ID,
    session_id=SESSION_ID,
    new_message=user_input,
):
    print("*** is_final_response:", event.is_final_response())
    if event.content and event.content.parts:
        for part in event.content.parts:
            print(part)
    print("=" * 100)

*** is_final_response: False
video_metadata=None thought=None inline_data=None file_data=None thought_signature=b'\n\xe6\x04\x01T\xa8\\\xeeR\x01\rK\xb6\x81\xc7\xb9\xd1\xec\xbd\xf1\r\x94\x98#3\xa7\xe7\x92j\x00\xe1\x90 W\xea\x0b\xbd\\\x02\xc81\xaa\x83y\x8c\xc1\t\x07\r\x94\x86\x05\xe4\xad \xa6|3jO\x14\xaa\x91\xaf\xbd\xb3\xe2\x16M\x88\'\xb0\x93A\xad\xdd\xfd.j\xd8\xc0\xfbtw\x85w ?\x1a\xd2\x98\xfe\x17X\x84\xe4w\xc6\n\x06\x99\x94|\xe2\xd0\xf2\x9f\xeb\x1f9\x16r\xe2GGX\xe2s\x17D\x8fsY\xc6\x93\x93W\xbcQ-&p\xd9\x91\xc2G\xc6/2\xd8\x85>\xc7\x9b\xdfu\xaaNko\xf2\xee\x1fS\x0c\x9c~\xd2\xc3y\xfc+\xa4\x17"\x047ygK\xc5D\xe3\xf2\x9fs\x16\x819d\xb7<T\xeb\x07|I\xac\xe0\xc0\xa0\xdb\x82\x7f\xa0n\x06\xbd\xd0\xef\xc0rv_c?\xc2"\xe8Z\x90\x02\x0c-\xb2\xfd\x92\xbc>\x8f\xc0\xa5\xd1\x8c\xb1\r>\xda\xf3\xba\x19\xd5\xa0a<\xc1\x9e\xad\x1f\x1ch\x8f\xef\xca\xfe\x1b[\xde A\xca\xb5\x11\x92\x17\x8a#1\x04\xbc\x82\n\xe6<\x18DG\xb8\xdb\xce\x81}~\x1c\x7f\xbbDr\xdf\x1a*_\x96\xe40\x18\xabUo{\x95K\xa3\x00\xaaS\xe9\xfac\x8b\xb1\xe2\xd

In [31]:
query = "私を森さんと呼んで"
user_input = types.Content(role="user", parts=[types.Part(text=query)])

async for event in runner.run_async(
    user_id=USER_ID,
    session_id=SESSION_ID,
    new_message=user_input,
):
    print("*** is_final_response:", event.is_final_response())
    if event.content and event.content.parts:
        for part in event.content.parts:
            print(part)
    print("=" * 100)

*** is_final_response: False
video_metadata=None thought=None inline_data=None file_data=None thought_signature=b"\n\xf6\x01\x01T\xa8\\\xee\xd4\xc1\xfa\xde\x11\rY7M*~#\x954\x05\x92\xb9+\x15\xcf\xd4q\t\xc2d\xa9\xb7\xa5\xee\x83\xb9PU\xd5\x0fo\xe0\xc8\x1b\xdb\x8d*H\xfdh\xde\xfb\xedF\xb4\xe90#\xc8\xfc\xda\xe9\xcd\x9f\xa7 uM)\xec6\xdeY\xc1p\x11\xc1QI'\xaa<\xe4\xe1\xf7\x17\x11\xda\x8d\xea\x07\n\xb4\x13\x1fw\xa9+[}?Y6\xbc\x97Z\x7f\x96\x14\x88y\xee\xbe\x19\x84\xde\xe6\x05\x80\x1d\xaa\x80\x912t\x1eb*\x1c[\x85\xfc\xb3O\x9cp\x16J\xfe\xa3\x89\xf8\xb7\x8c\xbb\xc5L\xb6\x93S\xfd\x9f\xbd\xb9=\xcb\xc5\xf3\x8a\xdc\xfb~%y\x89F\x01\xbdY\xf1z\x14\x89:\xf2\xa2\x90\x87f\xc3\x1d\xeb)\xdf\xf49\n\xab,\xd1aj=\x81\xb4\xf0\x91\x15%\xc7*\xe3\xe4\x97\xfc\xc2\xec$t\xbd\x81 \x86<\x8d\xa6\xca\xf4/B-Y\xf2i/y\xf07Z`\xb0b<\x1c\xb4\x87\xfa\xee\xe8{\x9e\x11o\xd5\xb0\xf6" code_execution_result=None executable_code=None function_call=FunctionCall(
  args={
    'name': '森さん'
  },
  id='adk-87616a7b-d045-4c83-a727-4f96ace323ea'

In [32]:
query = "現在のリマインダーは？"
user_input = types.Content(role="user", parts=[types.Part(text=query)])

async for event in runner.run_async(
    user_id=USER_ID,
    session_id=SESSION_ID,
    new_message=user_input,
):
    print("*** is_final_response:", event.is_final_response())
    if event.content and event.content.parts:
        for part in event.content.parts:
            print(part)
    print("=" * 100)

*** is_final_response: False
video_metadata=None thought=None inline_data=None file_data=None thought_signature=b'\n\x8c\x01\x01T\xa8\\\xee\x18\x94N\xf1\x0c\xa5E\xf9F\xf5\xa7*\x10\xaf\x9c\xa7\xee\xc5\x04\xb9\x01\x0f\xcb\xa4R\xdd\x0cjI\x17\x8da\xf6\x06\x93\xe0P\xa5\xe1\x85\xb5\xcd\x1br\xfbuv=\xf8\x05\\\x8f\\lB\xba\xb5TyJ\xa5\xc1-\x0c0\xe2\x19s7\xc5\xb5\xd2\xa2\xaa\nx\x11\xc1\xf1\xe5}\xa1\xc9\xb7!\x9d\x9a\x9eU\xc7\x0f\x1foj\xb4@\xb5\x06\xca\xd0\xc6-]h\x10\x01\xb9\r\x9b\x91\xa5\\\xed\x89\xfef\x15\xd0\x9d60Mst\xfb\x149\xe3\xd7\x1c.\xf2>3\x82' code_execution_result=None executable_code=None function_call=FunctionCall(
  args={},
  id='adk-acfabd70-7e1b-4358-8c89-d47604a59c7f',
  name='view_reminders'
) function_response=None text=None
*** is_final_response: False
video_metadata=None thought=None inline_data=None file_data=None thought_signature=None code_execution_result=None executable_code=None function_call=None function_response=FunctionResponse(
  id='adk-acfabd70-7e1b-4358-8c89-d47604